In [ ]:

!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



## *Reading Time Series Data from Files*, 

This chapter is an introduction to time series data. This chapter shows you how to read data from various and commonly used file
types, whether stored locally or on the cloud. The recipes will highlight advanced options
for ingesting, preparing, and transforming data into a time series DataFrame for
later analysis.  

We will cover the following recipes on how to ingest data into a pandas DataFrame:
* Reading data from CSVs and other delimited files
* Reading data from an Excel file
* Reading data from URLs


# Recipe 1: Reading from CSVs and Other Delimited Files
In this recipe, you will use the pandas.read_csv() function, which offers a large set
of parameters that you will explore to ensure the data is properly read into a time series
DataFrame. In addition, you will learn how to specify an index column, parse the index
to be of the type DatetimeIndex, and parse string columns that contain dates into
datetime objects

In [ ]:
import pandas as pd
from pathlib import Path
pd.__version__

'1.3.5'

In [ ]:
filepath = Path('./movieboxoffice.csv')

The first column in the CSV file contains movie release dates, and it needs to
be set as an index of type DatetimeIndex (index_col=0 and parse_
dates=['Date']). Specify which columns you want to include by providing a list
of column names to usecols. The default behavior is that the first row includes the
header (header=0):

In [ ]:
ts = pd.read_csv(filepath,
                 header=0,
             
                 parse_dates=['Date'],
                 index_col=0,
                 usecols=['Date',
                          'DOW',
                          'Daily',
                          'Forecast',
                          'Percent Diff'])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


Print a summary of the DataFrame to check the index and column data types:

In [ ]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DOW           128 non-null    object
 1   Daily         128 non-null    object
 2   Forecast      128 non-null    object
 3   Percent Diff  128 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


**Data Cleanup**
Notice that the Date column is now an index (not a column) of type
DatetimeIndex. Additionally, both Daily and Forecast columns have the
wrong dtype inference. You would expect them to be of type float. The issue is
due to the source CSV file containing dollar signs ($) and thousand separators (,)
in both columns.To fix this, you need to remove both the dollar sign ($) and thousand separators (,)
or any non-numeric character. You can accomplish this using str.replace(),
which can take a regular expression to remove all non-numeric characters but
exclude the period (.) for the decimal place. Removing these characters does not
convert the dtype, so you will need to cast those two columns as a float dtype
using .astype(float)

In [ ]:
clean = lambda x: x.str.replace('[^\d]', '', regex=True)                                                      
c_df = ts[['Daily', 'Forecast']].apply(clean, axis=1)
ts[['Daily', 'Forecast']] = c_df.astype(float)
                                

In [ ]:
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,12578989.0,23503646.0,-46.48%
2021-04-27,Saturday,9937401.0,19762255.0,-49.72%
2021-04-28,Sunday,8220316.0,11699126.0,-29.74%
2021-04-29,Monday,3353026.0,6665265.0,-49.69%
2021-04-30,Tuesday,3010524.0,3482819.0,-13.56%


In [ ]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOW           128 non-null    object 
 1   Daily         128 non-null    float64
 2   Forecast      128 non-null    float64
 3   Percent Diff  128 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


To get the exact memory usage for each column, including the index, you can
use the memory_usage() method:

In [ ]:
ts.memory_usage()

Index           1024
DOW             1024
Daily           1024
Forecast        1024
Percent Diff    1024
dtype: int64

In [ ]:
ts.memory_usage().sum()

5120

So far, you have used a few of the available parameters when reading a CSV file using
read_csv.

## Using `date_parser`

In [ ]:
ts.dtypes

DOW              object
Daily           float64
Forecast        float64
Percent Diff     object
dtype: object

You can also pass a lambda function that uses the to_datetime function in
pandas to date_parser. You can specify the string representation for the date format
inside to_datetime(), as demonstrated in the following code:
The preceding code will print out the first five rows of the ts DataFrame displaying
a properly parsed Date index.

In [ ]:
date_parser = lambda x: pd.to_datetime(x, format="%d-%b-%y")
ts = pd.read_csv(filepath,
                 parse_dates=[0],
                 index_col=0,
                 date_parser=date_parser,
                 usecols=[0,1,3, 7, 6])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


Let's break it down. In the preceding code, you passed two arguments to the to_
datetime function: the object to convert to datetime and an explicit format string.
Since the date is stored as a string in the form 26-Apr-21, you passed "%d-%b-%y" to
reflect that:
* %d represents the day of the month, such as 01 or 02.
* %b represents the abbreviated month name, such as Apr or May.
* %y represents a two-digit year, such as 19 or 20.

# Recipe 2: Reading data from an Excel file
In this recipe, you will be using the pandas.read_excel() function and examining 
the various parameters available to ensure the data is read properly as a DataFrame with 
a DatetimeIndex for time series analysis. In addition, you will explore different options 
to read Excel files with multiple sheets

* To install openpyxl using conda, run the following command in the terminal:

```
conda install openpyxl
```
* To install using pip, run the following command:

```
pip install openpyxl
```

In [ ]:
#!conda install openpyxl -y
#!pip install openpyxl

Import the libraries for this recipe
Read the Excel (.xlxs) file using the read_excel()function. 

In [ ]:
import pandas as pd
from pathlib import Path
filepath = Path('../../datasets/Ch2/sales_trx_data.xlsx')

In [ ]:
import openpyxl
openpyxl.__version__

'3.0.9'

In [ ]:
excelfile = pd.ExcelFile(filepath)
excelfile.sheet_names

['2017', '2018']

In [ ]:
excelfile.parse('2017')

,Line_Item_ID,Date,Credit_Card_Number,Quantity,Menu_Item
0,1,2017-01-01,7437926611570790,1,spicy miso ramen
1,2,2017-01-01,7437926611570790,1,spicy miso ramen
2,3,2017-01-01,8421920068932810,3,tori paitan ramen
3,4,2017-01-01,8421920068932810,3,tori paitan ramen
4,5,2017-01-01,4787310681569640,1,truffle butter ramen
...,...,...,...,...,...
36759,36760,2017-12-31,3149176814183170,4,vegetarian curry + king trumpet mushroom ramen
36760,36761,2017-12-31,2005206728506200,1,tonkotsu ramen
36761,36762,2017-12-31,2005206728506200,1,tonkotsu ramen
36762,36763,2017-12-31,1130414909788170,2,soft-shell miso crab ramen


 The sales data will be partitioned (split) by year, where each sheet contained sales for 
a particular year. In this case, concatenating the two DataFrames is a natural choice. The 
pandas.concat() function is like the DataFrame.append() function, in which the 
second DataFrame was added (appended) to the end of the first DataFrame.

In [ ]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=[0,1],
                    parse_dates=True)
ts.keys()

dict_keys([0, 1])

In [ ]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=['2017','2018'],
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [ ]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=None,
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [ ]:
ts['2017'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36764 entries, 2017-01-01 to 2017-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        36764 non-null  int64 
 1   Credit_Card_Number  36764 non-null  int64 
 2   Quantity            36764 non-null  int64 
 3   Menu_Item           36764 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.4+ MB


In [ ]:
ts['2018'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37360 entries, 2018-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        37360 non-null  int64 
 1   Credit_Card_Number  37360 non-null  int64 
 2   Quantity            37360 non-null  int64 
 3   Menu_Item           37360 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.4+ MB


In [ ]:
ts_combined = pd.concat([ts['2017'],ts['2018']])

In [ ]:
ts_combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74124 entries, 2017-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        74124 non-null  int64 
 1   Credit_Card_Number  74124 non-null  int64 
 2   Quantity            74124 non-null  int64 
 3   Menu_Item           74124 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.8+ MB


In [ ]:
pd.concat(ts).index

MultiIndex([('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ...
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31')],
           names=[None, 'Date'], length=74124)

To reduce the number of levels, you can use the droplevel(level=0) method 
to drop the first level after pandas .concat() shown as follows

In [ ]:
ts_combined = pd.concat(ts).droplevel(level=0)
ts_combined.head()

,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


In [ ]:
ts = pd.read_excel(filepath,
                   index_col=1,
                   sheet_name='2018',
                   parse_dates=True)
type(ts)

pandas.core.frame.DataFrame

## There is more
* Using `ExcelFile`

In [ ]:
excelfile = pd.ExcelFile(filepath)
excelfile.parse(sheet_name='2017',
                index_col=1,
                parse_dates=True).head()


,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


# Recipe 3: Reading data from a URL
* Many of the pandas reader functions can read data from remote locations by passing 
a URL path. 
* For example, both read_csv() and read_excel() can take a URL to 
read a file that is accessible via the internet. In this recipe, you will read a CSV file using 
pandas.read_csv() and Excel files using pandas.read_excel() from remote 
locations, such as GitHub and AWS S3 (private and public buckets). 
* You will also read data directly from an HTML page into a pandas DataFrame

In [ ]:
#!conda install boto3 s3fs html5lib lxml -y
#!pip install boto3 s3fs html5lib lxml

* To install using pip, you can use the following command:

```
pip install boto3 s3fs lxml
```

* To install using Conda, you can use:

```
conda install boto3 s3fs lxml -y
```

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import  lxml
print(f'''
pandas -> {pd.__version__}
lxml -> {lxml.__version__}
''')


pandas -> 1.3.4
boto3 -> 1.20.24
s3fs -> 2022.01.0
lxml -> 4.8.0



## Reading from GitHub
* This is most important part since you'll be using it frequently in your project to read the data from the project's github page
*To read a CSV file from GitHub, you will need the URL to the raw content. If you 
copy the file's GitHub URL from the browser and use it as the file path, you will get 
a URL that looks like this: https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv  This URL is a pointer to the web page in GitHub and 
not the data itself; hence when using pd.read_csv(), it will throw an error

In [ ]:
# example of produced error
url = 'https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv'
# pd.read_csv(url)


Instead, you will need the raw content, which will give you a URL that 
looks like this:
The first column in the file is the Date column. You will need to parse (parse_
date parameter) and convert it to DatetimeIndex (index_col parameter).

In [ ]:
url = 'https://github.com/FuTSA23/time-series-analysis-datasets/blob/main/AirQualityUCI.csv'
date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%Y")

df = pd.read_csv(url,
                 delimiter=';',
                 index_col='Date',
                 date_parser=date_parser)

df.iloc[:3,1:4]

,CO(GT),PT08.S1(CO),NMHC(GT)
Date,,,
2004-03-10,2.6,1360.00,150
2004-03-10,2.0,1292.25,112
2004-03-10,2.2,1402.00,88


## Reading from HTML
pandas offers an elegant way to read HTML tables and convert the content into a pandas 
DataFrame using the pandas.read_html() function: 
* In the following recipe, we will extract HTML tables from Wikipedia for COVID-19 
pandemic tracking cases by country and by territory (https://en.wikipedia.
org/wiki/COVID-19_pandemic_by_country_and_territory)

In [ ]:
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
results = pd.read_html(url)
print(len(results))


71


In [ ]:
# for i, k in enumerate(results):
#     print(i)
#     display(k.head())
    

In [ ]:
df = results[15]
df.columns

Index(['Region[28]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[29]'],
      dtype='object')

In [ ]:
df[['Total cases', 'Total deaths', 'Cases per million']].head()

,Total cases,Total deaths,Cases per million
0,141222709,1090244,315709
1,87015319,1041592,235937
2,51658794,478860,221103
3,57294104,1272395,133281
4,22107880,420389,93363


## Example how `read_html()` works

In [ ]:
import pandas as pd
html = """
 <table>
   <tr>
     <th>Ticker</th>
     <th>Price</th>
   </tr>
   <tr>
     <td>MSFT</td>
     <td>230</td>
   </tr>
   <tr>
     <td>APPL</td>
     <td>300</td>
   </tr>
     <tr>
     <td>MSTR</td>
     <td>120</td>
   </tr>
 </table>

 </body>
 </html>
 """

df = pd.read_html(html)
df[0]


,Ticker,Price
0,MSFT,230
1,APPL,300
2,MSTR,120


In the preceding code, the read_html() function parsed the HTML code and converted 
the HTML table into a pandas DataFrame. The headers between the <th> and </th>
tags represent the column names of the DataFrame, and the content between the <tr></
td> and </td></tr> tags represent the row data of the DataFrame. 

## Using `attr` option in `read.html()`
The read_html() function has an optional attr argument, which takes a dictionary 
of valid HTML <table> attributes, such as id or class. For example, you can use 
the attr parameter to narrow down the tables returned to those that match the class
attribute sortable as in <table class="sortable">. The read_html function 
will inspect the entire HTML page to ensure you target the right set of attributes.




In [2]:
#!conda install html5lib beautifulSoup4
#!pip install html5lib beautifulSoup4

In [ ]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
df = pd.read_html(url, attrs={'class': 'sortable'})
len(df)

7

In [ ]:
df[3].columns

Index(['Region[28]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[29]'],
      dtype='object')